In [7]:
import tkinter as tk
from tkinter import *
import nltk
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import random
import pickle
import json
import os
import spacy
import re

nlp = spacy.load('es_core_news_sm')

with open('intents_copy.json', encoding='utf-8') as file:
    data = json.load(file)

def lemmatize_sentence(sentence):
    sentence = sentence.lower()
    sentence = re.sub('[^a-zA-ZáéíóúÁÉÍÓÚñÑüÜ]', ' ', sentence)  # Excluye caracteres no deseados
    doc = nlp(sentence)
    lemmas = [token.lemma_ for token in doc if token.lemma_ != "-PRON-"]  # Excluye pronombres
    return lemmas

# Eliminar archivos antiguos si existen
if os.path.exists("data.pickle"):
    os.remove("data.pickle")
if os.path.exists("model.keras"):
    os.remove("model.keras")

print("Archivos antiguos eliminados, procesando datos y entrenando el modelo desde cero")

words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        docs_x.append(wrds)
        docs_y.append(intent['tag'])

        words.extend(lemmatize_sentence(pattern))

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

words = sorted(list(set(words)))
labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    wrds = lemmatize_sentence(' '.join(doc))

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

training = np.array(training)
output = np.array(output)

with open("data.pickle", "wb") as f:
    pickle.dump((words, labels, training, output), f)

model = Sequential()
model.add(Dense(8, input_shape=(len(training[0]),), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(len(output[0]), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(training, output, epochs=3000, batch_size=8, verbose=1)
model.save("model_chatbot.keras")

def bag_of_words(s, words):
    s = s.lower()
    s = re.sub('[^a-zA-ZáéíóúÁÉÍÓÚñÑüÜ]', ' ', s)  # Se reemplaza por espacio cualquier caracter que no sea letras en español
    bag = [0 for _ in range(len(words))]
    s_words = lemmatize_sentence(s)

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return np.array(bag)

def chatbot_response(msg):
    bow = bag_of_words(msg, words)
    results = model.predict(np.array([bow]))[0]
    results_index = np.argmax(results)
    tag = labels[results_index]

    for tg in data['intents']:
        if tg['tag'] == tag:
            responses = tg['responses']
            return random.choice(responses)

    return "Lo siento, no entiendo lo que quieres decir."


Archivos antiguos eliminados, procesando datos y entrenando el modelo desde cero
Epoch 1/3000
9/9 [==============================] - 0s 729us/step - loss: 2.4673 - accuracy: 0.1538
Epoch 2/3000
9/9 [==============================] - 0s 1ms/step - loss: 2.4602 - accuracy: 0.1846
Epoch 3/3000
9/9 [==============================] - 0s 996us/step - loss: 2.4546 - accuracy: 0.1692
Epoch 4/3000
9/9 [==============================] - 0s 847us/step - loss: 2.4483 - accuracy: 0.1692
Epoch 5/3000
9/9 [==============================] - 0s 820us/step - loss: 2.4426 - accuracy: 0.1692
Epoch 6/3000
9/9 [==============================] - 0s 1ms/step - loss: 2.4364 - accuracy: 0.1385
Epoch 7/3000
9/9 [==============================] - 0s 883us/step - loss: 2.4289 - accuracy: 0.1692
Epoch 8/3000
9/9 [==============================] - 0s 1ms/step - loss: 2.4212 - accuracy: 0.1692
Epoch 9/3000
9/9 [==============================] - 0s 2ms/step - loss: 2.4136 - accuracy: 0.1692
Epoch 10/3000
9/9 [========

9/9 [==============================] - 0s 728us/step - loss: 0.4644 - accuracy: 0.9538
Epoch 83/3000
9/9 [==============================] - 0s 751us/step - loss: 0.4404 - accuracy: 0.9846
Epoch 84/3000
9/9 [==============================] - 0s 706us/step - loss: 0.4180 - accuracy: 0.9692
Epoch 85/3000
9/9 [==============================] - 0s 694us/step - loss: 0.3995 - accuracy: 0.9846
Epoch 86/3000
9/9 [==============================] - 0s 708us/step - loss: 0.3809 - accuracy: 0.9846
Epoch 87/3000
9/9 [==============================] - 0s 723us/step - loss: 0.3632 - accuracy: 0.9846
Epoch 88/3000
9/9 [==============================] - 0s 695us/step - loss: 0.3448 - accuracy: 0.9846
Epoch 89/3000
9/9 [==============================] - 0s 762us/step - loss: 0.3290 - accuracy: 0.9846
Epoch 90/3000
9/9 [==============================] - 0s 680us/step - loss: 0.3131 - accuracy: 0.9846
Epoch 91/3000
9/9 [==============================] - 0s 823us/step - loss: 0.2997 - accuracy: 0.9846
Epoc

9/9 [==============================] - 0s 836us/step - loss: 0.0237 - accuracy: 1.0000
Epoch 163/3000
9/9 [==============================] - 0s 863us/step - loss: 0.0232 - accuracy: 1.0000
Epoch 164/3000
9/9 [==============================] - 0s 908us/step - loss: 0.0227 - accuracy: 1.0000
Epoch 165/3000
9/9 [==============================] - 0s 850us/step - loss: 0.0222 - accuracy: 1.0000
Epoch 166/3000
9/9 [==============================] - 0s 880us/step - loss: 0.0217 - accuracy: 1.0000
Epoch 167/3000
9/9 [==============================] - 0s 711us/step - loss: 0.0213 - accuracy: 1.0000
Epoch 168/3000
9/9 [==============================] - 0s 777us/step - loss: 0.0208 - accuracy: 1.0000
Epoch 169/3000
9/9 [==============================] - 0s 736us/step - loss: 0.0203 - accuracy: 1.0000
Epoch 170/3000
9/9 [==============================] - 0s 757us/step - loss: 0.0199 - accuracy: 1.0000
Epoch 171/3000
9/9 [==============================] - 0s 697us/step - loss: 0.0195 - accuracy: 1.

9/9 [==============================] - 0s 644us/step - loss: 0.0060 - accuracy: 1.0000
Epoch 243/3000
9/9 [==============================] - 0s 695us/step - loss: 0.0059 - accuracy: 1.0000
Epoch 244/3000
9/9 [==============================] - 0s 642us/step - loss: 0.0058 - accuracy: 1.0000
Epoch 245/3000
9/9 [==============================] - 0s 666us/step - loss: 0.0057 - accuracy: 1.0000
Epoch 246/3000
9/9 [==============================] - 0s 603us/step - loss: 0.0056 - accuracy: 1.0000
Epoch 247/3000
9/9 [==============================] - 0s 618us/step - loss: 0.0056 - accuracy: 1.0000
Epoch 248/3000
9/9 [==============================] - 0s 634us/step - loss: 0.0055 - accuracy: 1.0000
Epoch 249/3000
9/9 [==============================] - 0s 602us/step - loss: 0.0054 - accuracy: 1.0000
Epoch 250/3000
9/9 [==============================] - 0s 637us/step - loss: 0.0053 - accuracy: 1.0000
Epoch 251/3000
9/9 [==============================] - 0s 588us/step - loss: 0.0053 - accuracy: 1.

9/9 [==============================] - 0s 742us/step - loss: 0.0024 - accuracy: 1.0000
Epoch 323/3000
9/9 [==============================] - 0s 690us/step - loss: 0.0024 - accuracy: 1.0000
Epoch 324/3000
9/9 [==============================] - 0s 769us/step - loss: 0.0024 - accuracy: 1.0000
Epoch 325/3000
9/9 [==============================] - 0s 740us/step - loss: 0.0024 - accuracy: 1.0000
Epoch 326/3000
9/9 [==============================] - 0s 732us/step - loss: 0.0023 - accuracy: 1.0000
Epoch 327/3000
9/9 [==============================] - 0s 728us/step - loss: 0.0023 - accuracy: 1.0000
Epoch 328/3000
9/9 [==============================] - 0s 707us/step - loss: 0.0023 - accuracy: 1.0000
Epoch 329/3000
9/9 [==============================] - 0s 700us/step - loss: 0.0023 - accuracy: 1.0000
Epoch 330/3000
9/9 [==============================] - 0s 756us/step - loss: 0.0022 - accuracy: 1.0000
Epoch 331/3000
9/9 [==============================] - 0s 726us/step - loss: 0.0022 - accuracy: 1.

...

9/9 [==============================] - 0s 724us/step - loss: 3.6680e-09 - accuracy: 1.0000
Epoch 2693/3000
9/9 [==============================] - 0s 810us/step - loss: 5.5020e-09 - accuracy: 1.0000
Epoch 2694/3000
9/9 [==============================] - 0s 740us/step - loss: 5.5020e-09 - accuracy: 1.0000
Epoch 2695/3000
9/9 [==============================] - 0s 788us/step - loss: 5.5020e-09 - accuracy: 1.0000
Epoch 2696/3000
9/9 [==============================] - 0s 719us/step - loss: 7.3360e-09 - accuracy: 1.0000
Epoch 2697/3000
9/9 [==============================] - 0s 767us/step - loss: 9.1699e-09 - accuracy: 1.0000
Epoch 2698/3000
9/9 [==============================] - 0s 741us/step - loss: 7.3360e-09 - accuracy: 1.0000
Epoch 2699/3000
9/9 [==============================] - 0s 765us/step - loss: 3.6680e-09 - accuracy: 1.0000
Epoch 2700/3000
9/9 [==============================] - 0s 720us/step - loss: 9.1699e-09 - accuracy: 1.0000
Epoch 2701/3000
9/9 [==============================] 

9/9 [==============================] - 0s 777us/step - loss: 5.5020e-09 - accuracy: 1.0000
Epoch 2769/3000
9/9 [==============================] - 0s 714us/step - loss: 5.5020e-09 - accuracy: 1.0000
Epoch 2770/3000
9/9 [==============================] - 0s 756us/step - loss: 9.1699e-09 - accuracy: 1.0000
Epoch 2771/3000
9/9 [==============================] - 0s 720us/step - loss: 1.1004e-08 - accuracy: 1.0000
Epoch 2772/3000
9/9 [==============================] - 0s 767us/step - loss: 1.6506e-08 - accuracy: 1.0000
Epoch 2773/3000
9/9 [==============================] - 0s 701us/step - loss: 1.1004e-08 - accuracy: 1.0000
Epoch 2774/3000
9/9 [==============================] - 0s 749us/step - loss: 1.8340e-08 - accuracy: 1.0000
Epoch 2775/3000
9/9 [==============================] - 0s 684us/step - loss: 1.4672e-08 - accuracy: 1.0000
Epoch 2776/3000
9/9 [==============================] - 0s 740us/step - loss: 9.1699e-09 - accuracy: 1.0000
Epoch 2777/3000
9/9 [==============================] 

9/9 [==============================] - 0s 726us/step - loss: 1.4672e-08 - accuracy: 1.0000
Epoch 2845/3000
9/9 [==============================] - 0s 766us/step - loss: 7.3360e-09 - accuracy: 1.0000
Epoch 2846/3000
9/9 [==============================] - 0s 894us/step - loss: 9.1699e-09 - accuracy: 1.0000
Epoch 2847/3000
9/9 [==============================] - 0s 845us/step - loss: 1.1004e-08 - accuracy: 1.0000
Epoch 2848/3000
9/9 [==============================] - 0s 990us/step - loss: 9.1699e-09 - accuracy: 1.0000
Epoch 2849/3000
9/9 [==============================] - 0s 1ms/step - loss: 9.1699e-09 - accuracy: 1.0000
Epoch 2850/3000
9/9 [==============================] - 0s 1ms/step - loss: 7.3360e-09 - accuracy: 1.0000
Epoch 2851/3000
9/9 [==============================] - 0s 1ms/step - loss: 9.1699e-09 - accuracy: 1.0000
Epoch 2852/3000
9/9 [==============================] - 0s 1ms/step - loss: 5.5020e-09 - accuracy: 1.0000
Epoch 2853/3000
9/9 [==============================] - 0s 1ms

9/9 [==============================] - 0s 1ms/step - loss: 1.0424e-05 - accuracy: 1.0000
Epoch 2922/3000
9/9 [==============================] - 0s 1ms/step - loss: 1.0081e-05 - accuracy: 1.0000
Epoch 2923/3000
9/9 [==============================] - 0s 1ms/step - loss: 9.8096e-06 - accuracy: 1.0000
Epoch 2924/3000
9/9 [==============================] - 0s 1ms/step - loss: 9.4869e-06 - accuracy: 1.0000
Epoch 2925/3000
9/9 [==============================] - 0s 986us/step - loss: 9.2595e-06 - accuracy: 1.0000
Epoch 2926/3000
9/9 [==============================] - 0s 933us/step - loss: 8.9752e-06 - accuracy: 1.0000
Epoch 2927/3000
9/9 [==============================] - 0s 837us/step - loss: 8.7625e-06 - accuracy: 1.0000
Epoch 2928/3000
9/9 [==============================] - 0s 857us/step - loss: 8.5131e-06 - accuracy: 1.0000
Epoch 2929/3000
9/9 [==============================] - 0s 875us/step - loss: 8.3169e-06 - accuracy: 1.0000
Epoch 2930/3000
9/9 [==============================] - 0s 695

9/9 [==============================] - 0s 632us/step - loss: 3.1526e-06 - accuracy: 1.0000
Epoch 2998/3000
9/9 [==============================] - 0s 625us/step - loss: 3.1159e-06 - accuracy: 1.0000
Epoch 2999/3000
9/9 [==============================] - 0s 645us/step - loss: 3.0811e-06 - accuracy: 1.0000
Epoch 3000/3000
9/9 [==============================] - 0s 586us/step - loss: 3.0536e-06 - accuracy: 1.0000


In [8]:
# Cargar el modelo Keras
model = tf.keras.models.load_model("model_chatbot.keras")

    # Crear el convertidor de TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
    
    # Configurar opciones del convertidor
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float32]  
    
    # Convertir el modelo
tflite_model = converter.convert()

    # Guardar el modelo TFLite en un archivo
with open("model_chatbot.tflite", "wb") as f:
    f.write(tflite_model)

print("Modelo convertido a TFLite y guardado.")

INFO:tensorflow:Assets written to: /var/folders/v9/712wq54s0w93bv64kl9rdlvh0000gn/T/tmp22_edplm/assets


INFO:tensorflow:Assets written to: /var/folders/v9/712wq54s0w93bv64kl9rdlvh0000gn/T/tmp22_edplm/assets


Modelo convertido a TFLite y guardado.


2024-07-11 15:02:08.240084: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-11 15:02:08.240096: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-11 15:02:08.240227: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/v9/712wq54s0w93bv64kl9rdlvh0000gn/T/tmp22_edplm
2024-07-11 15:02:08.240864: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-11 15:02:08.240870: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/v9/712wq54s0w93bv64kl9rdlvh0000gn/T/tmp22_edplm
2024-07-11 15:02:08.242112: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-11 15:02:08.254993: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /var/folders/v9/712wq54s0w93bv64kl9rdlvh0000gn/T/tmp22_edplm
2024-07-

In [ ]:
# Configuración de la interfaz gráfica con Tkinter
base = Tk()  
base.title("Chatbot")  
base.geometry("400x500")  

ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font=("Arial", 12), wrap=WORD)
ChatLog.config(foreground="black")
ChatLog.insert(END, "SALUDOS BIENVENIDO\n\n") 
ChatLog.place(x=6, y=6, height=386, width=370) 

scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
scrollbar.place(x=376, y=6, height=386)

ChatLog.config(state=DISABLED)

EntryBox = Text(base, bd=0, bg="white", width="29", height="5", font=("Arial", 12), wrap=WORD)
EntryBox.place(x=6, y=401, height=90, width=265)

def send(event=None):
    msg = EntryBox.get("1.0", 'end-1c').strip()
    EntryBox.delete("0.0", END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="black")

        res = chatbot_response(msg)
        ChatLog.insert(END, "ChatBOT: " + res + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

SendButton = Button(base, font=("Verdana", 12, 'bold'), text="Send", width="9",
                   height=5, bd=0, bg="blue", activebackground="gold",
                   fg='#ffffff', command=send)
SendButton.place(x=282, y=401, height=90)

base.bind('<Return>', send)

base.mainloop()

In [4]:
print(chatbot_response("Mi planta tiene manchas bacterianas"))
print(chatbot_response("Mi planta tiene sarna"))
print(chatbot_response("No hay plagas en mi planta"))
print(chatbot_response("Mi planta tiene tizón tardío"))

1/1 [==============================] - 0s 45ms/step
Retira las hojas afectadas y asegúrate de que las herramientas de poda estén desinfectadas.
1/1 [==============================] - 0s 9ms/step
Asegúrate de que el suelo tenga un buen drenaje para evitar la acumulación de humedad.
1/1 [==============================] - 0s 10ms/step
Es excelente que no haya plagas. Mantén las buenas prácticas de cuidado.
1/1 [==============================] - 0s 10ms/step
Elimina las hojas y ramas afectadas y desinfecta las herramientas de poda.


In [5]:
# Sección añadida para identificar los intents que reconoce el modelo
def recognize_intents():
    intent_count = {intent['tag']: 0 for intent in data['intents']}
    
    test_sentences = [
        "Hola",  # saludo
        "Mi planta tiene manchas bacterianas",  # consejo sobre manchas bacterianas
        "Mi planta tiene sarna",  # consejo sobre sarna
        "No hay plagas en mi planta",  # estado de la planta
        "Mi planta tiene tizón tardío",  # consejo sobre tizón tardío
        "¿Qué tal?",  # saludo
        "Plaga de tizón temprano",  # consejo sobre tizón temprano
        "Mi planta está saludable",  # estado de la planta
        "Hay manchas en mi planta",  # consejo sobre manchas
        "Mi planta tiene roya"  # consejo sobre roya
    ]
    
    for sentence in test_sentences:
        bow = bag_of_words(sentence, words)
        results = model.predict(np.array([bow]))[0]
        results_index = np.argmax(results)
        tag = labels[results_index]

        for intent in data['intents']:
            if intent['tag'] == tag:
                intent_count[tag] += 1
    
    print("\nIntents reconocidos por el modelo:")
    for intent in intent_count:
        print(f"Intent: {intent}, Reconocido en {intent_count[intent]} pruebas")

# Llama a la función para ver los intents reconocidos
recognize_intents()

1/1 [==============================] - 0s 9ms/step

Intents reconocidos por el modelo:
Intent: saludos, Reconocido en 2 pruebas
Intent: mancha_bacteriana, Reconocido en 1 pruebas
Intent: sarampion_negro, Reconocido en 0 pruebas
Intent: podredumbre_negra, Reconocido en 0 pruebas
Intent: desconocido, Reconocido en 0 pruebas
Intent: tizon_temprano, Reconocido en 1 pruebas
Intent: tizon_tardio, Reconocido en 1 pruebas
Intent: quemadura_hoja, Reconocido en 0 pruebas
Intent: roya, Reconocido en 1 pruebas
Intent: sarna, Reconocido en 1 pruebas
Intent: sin_plaga, Reconocido en 2 pruebas
Intent: manchas, Reconocido en 1 pruebas


___